In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext(conf=conf)

1. 讀取檔案

In [2]:
data = sc.textFile('500input.txt')

2. 準備以下函數

2.1 reducer function 可以將相同Key的Value整理起來

In [3]:
def reducer1(x,y):
    return x+y

2.2 定義一個函數mapper將一開始讀進來的資料進行初步處理

若是屬於Ｍ（ｉ﹐ｊ）矩陣的元素就index j pop出來將資料擺成(j, (M, i, m))的形式，Ｎ（ｊ﹐ｋ）矩陣則是(j, (N, k, n))，寫成[list1]則是為了要將資料隔開。

In [4]:
def mapper(line):
    wordlist = line.split(",")
    maplist = []
    for i in range(0, len(wordlist), 5):
        list1 = wordlist[i:(i+4)]
        if list1[0]=='M':
            key = list1.pop(2)
            maplist.append((int(key),[list1]))
        else:
            key = list1.pop(1)
            maplist.append((int(key),[list1]))
    return maplist

2.3 mappermn可以搭配reducer1 function把資料的M矩陣和N矩陣元素分離

In [5]:
def mappermn(line):
    linecopy = line
    maplist = []
    for i in range(len(linecopy)):
        key = str(linecopy[i].pop(0))
        maplist.append((key, [linecopy[i]]))
    return maplist

2.4 mapper1 function是在兩個矩陣元素分離後使用，在每個Ｍ矩陣的元素後面都掛上整個Ｎ矩陣的元素方便後續運算。處理完後資料型態如下：

[(i1, m1, [(k1, n1), (k2, n2),...]), (i2, m2, [(k1, n1), (k2, n2),...])...]

以上資料形式搭配mapper2可以讓每個M矩陣的元素與N矩陣的元素都相乘到。

In [6]:
def mapper1(sepm):
    op = sepm
    for i in range(len(op)):
        op[i].append(sepn)
    return op

2.5 mapper2 function用了雙層迴圈把相同index j的所有(M, i, m)(N, k, n)資料轉換成((i, k), mn)。

In [7]:
def mapper2(op):
    maplist = []
    for i in range(len(op)):
        index1 = int(op[i][0])
        mul1 = int(op[i][1])
        for k in range(len(op[i][2])):
            index2 = int(op[i][2][k][0])
            mul2 = int(op[i][2][k][1])
            outcome = mul1*mul2
            key = (index1, index2)
            maplist.append((key, outcome))
    return maplist

3. 程式流程

3.1 用mapper function把資料轉成(j, (M, i, m))和(j, (N, k, n))的型態。

In [8]:
rdd0 = data.flatMap(mapper)

In [9]:
mn = rdd0.reduceByKey(reducer1).collect()

3.2 接下來的for loop針對每個index j 裡面的元素做運算，轉換成((i, k), mn)的形式。

In [10]:
ans_list = []
for j in range(len(mn)):
    
    #讀取index j的所有元素放入test
    test = mn[j][1]
    
    #以M,N當成key,mappermn搭配reducer1將test的資料成兩個資料集
    septest = mappermn(test)
    rdd_sep = sc.parallelize(septest).reduceByKey(reducer1)
    sepmn = rdd_sep.collect()
    sepn = sepmn[0][1]
    sepm = sepmn[1][1]
    
    #把M矩陣每個後面都直接加掛N矩陣的所有元素
    op = mapper1(sepm)
    
    #把處理好的資料放入mapper2轉換成((i, k), mn)的形式
    maplist = mapper2(op)
    
    
    ans_list.extend(maplist)

In [11]:
#資料排序
ans_list.sort()
ans_long = len(ans_list)

In [12]:
dim = len(data.collect())/2

接下來原本只要用以下兩行程式碼做最後一次map-reduce就可以

rdd_ans = sc.parallelize(loader)
answer = rdd_ans.reduceByKey(reducer1).collect()

不過直接將全部資料倒進去會出現 <outmemoryerror> 的訊息，所以要切割資料分批次做map-reduce

In [13]:
ans = []
for i in range(0, len(ans_list), int(ans_long/dim)):
    loader = ans_list[i:(i+int(ans_long/dim))]
    rdd_ans = sc.parallelize(loader)
    answer = rdd_ans.reduceByKey(reducer1).collect()
    ans.extend(answer)

Note: int(ans_long/dim)代表j的個數

In [14]:
#資料排序
ans.sort()

3.3 開一個新的檔案(mapreduce_output)將運算完的資料逐行印上去

In [15]:
file = open("mapreduce_output.txt", "w")
file.write("Output\n")
for i in range(len(ans)):
    file.write(str(ans[i][0][0]))
    file.write(',')
    file.write(str(ans[i][0][1]))
    file.write(',')
    file.write(str(ans[i][1]))
    file.write(' \n')
file.close()